# AJUSTE DE MODELOS

Una vez hemos explorado y preprocesado los datos estamos listos para ajustar los modelos. Debido a que no todos los modelos son adecuados para resolver los mismos problemas hemos seleccionado unos cuantos conjuntos de datos para resolver distintos problemas de Machine Learning que nos permitirán aplicar todos los algoritmos. Por supuesto, para cada caso concreto siempre habrá un algoritmo más adecuado, pero nuestro objetivo en este apartado es mostrar como se deben aplicar los algoritmos de Machine Learning usando MLlib y Pyspark.

In [95]:
from pyspark.sql import SparkSession
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, \
NaiveBayes, GBTClassifier, LinearSVC, MultilayerPerceptronClassifier
from pyspark.ml.clustering import KMeans, BisectingKMeans, GaussianMixture, PowerIterationClustering, LDA
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from ipynb.fs.full.Pipeline_Preprocesado import *

In [4]:
spark = SparkSession.builder.appName("Ajuste de Modelos").master("local[*]").getOrCreate()

# APRENDIZAJE SUPERVISADO

Primero analizaremos los siguientes algoritmos de Aprendizaje Supervisado: Árboles de Decisión, Bagging, Boosting, SVM, Regresión Logística, Multilayer Perceptron y Naive Bayes con el Dataset de detección de fraude que hemos explorado con anterioridad.

In [5]:
df = spark.read.csv('../../Datasets/Fraud_detection5.csv',header=True, inferSchema=True)

Cambiamos el nombre de la variable target, en este caso la variable "isFraud", a "target" para que funcione bien la Pipeline de preprocesado

In [6]:
df = df.withColumn("target", df["isFraud"]).drop("isFraud")

A continuación importaremos la pipeline de preprocesado que hemos definido con anterioridad para dejar los datos listos para poder analizarlos con los algoritmos de ML. Hay una consideración que es importante y es que no todos los algoritmos que vamos a ver necesitan tener los datos preprocesados de la misma forma. Por ejemplo, los árboles de decisión pueden trabajar perfectamente con variables categóricas sin necesidad de codificarlas, pero la regresión logística no. Como nuestro análisis intenta ser lo más extenso posible aplicaremos el mismo procesamiento a los datos para emplear cualquiera de los algoritmos. Adicionalmente, si quisieramos cambiar el método de imputación de los missings o el método de codificación de las variables categóricas tendriamos que ir al código que define la pipeline de preprocesao, aplicar los cambios y guardar la nueva pipeline.

In [7]:
pipeline_preprocesado = Pipeline.load("pipeline_preprocesado")

In [8]:
pipeline_model_preprocesado = pipeline_preprocesado.fit(df)

In [9]:
df = pipeline_model_preprocesado.transform(df)

Ahora ya estamos listos para aplicar los algoritmos de Machine Learning.

Lo primero que debemos hacer para poder introducir los datos en los modelos de Machine Learning de MLlib es procesarlos con un transformer llamado VectorAssembler. En el introduciremos las variables, menos el target, del conjunto de datos que deseamos introducir en el algoritmo y obtendremos una única de salida que daremos a los algoritmos.

In [10]:
assembler = VectorAssembler(
    inputCols=['step', 'amount', 'oldbalanceOrg', 'newbalanceOrg', 'oldbalanceDest', 'newbalanceDest', 
              'isFlaggedFraud', 'type-encoded', 'nameOrig-encoded', 'nameDest-encoded'],
    outputCol="all_features"
)
df = assembler.transform(df)

Lo segundo es dividir nuestros datos en train y test.

In [11]:
datos_entrenamiento, datos_test = df.randomSplit([0.8,0.2])

1) Regresión Logística

In [12]:
lr = LogisticRegression(featuresCol='all_features', labelCol='target',maxIter=60, elasticNetParam = 0.5)
Model_lr = lr.fit(datos_entrenamiento)

22/07/05 00:56:19 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:56:22 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:56:24 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/07/05 00:56:24 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/07/05 00:56:24 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/07/05 00:56:24 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/07/05 00:56:25 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:56:26 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:56:26 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:56:27 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:56:28 WAR

Ahora podemos realizar predicciones sobre el conjunto de test y sacar distintas métricas que midan su calidad.

In [13]:
prediccion = Model_lr.transform(datos_test)

Ahora vamos a cambiar el tipo de la columna "target" para evitar errores a la hora de obtener las métricas del ajuste.

In [14]:
prediccion= prediccion.withColumn("target", prediccion["target"].cast("double"))
prediccion = prediccion.select("target", "prediction")

In [15]:
metrics = MulticlassMetrics(prediccion.rdd.map(tuple))

/home/lu/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
22/07/05 00:56:59 WARN DAGScheduler: Broadcasting large task binary with size 22.0 MiB


In [16]:
f_1 = metrics.fMeasure(1.0)
print(f'La métrica f_1 es: {f_1}')

22/07/05 00:57:02 WARN DAGScheduler: Broadcasting large task binary with size 22.0 MiB


La métrica f_1 es: 0.012941176470588235


In [17]:
precision = metrics.precision(1.0)
print(f'La métrica Precisión es: {precision}')

La métrica Precisión es: 0.00684931506849315


In [18]:
recall = metrics.recall(1.0)
print(f'La métrica Recall es: {recall}')

La métrica Recall es: 0.11702127659574468


2) Árbol de decisión

In [19]:
dt = DecisionTreeClassifier(featuresCol = 'all_features', labelCol = 'target', maxDepth = 7,
                                     impurity='gini')
Model_dt = dt.fit(datos_entrenamiento)

22/07/05 00:57:05 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:57:08 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:57:10 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:57:11 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 00:57:13 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/07/05 00:57:14 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 00:57:15 WARN MemoryStore: Not enough space to cache rdd_487_0 in memory! (computed 229.9 MiB so far)
22/07/05 00:57:15 WARN BlockManager: Persisting block rdd_487_0 to disk instead.
22/07/05 00:58:58 WARN MemoryStore: Not enough space to cache rdd_487_0 in memory! (computed 229.9 MiB so far)
22/07/05 01:00:17 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 01:00:19 WARN MemoryStore: Not enough space to cache rdd_487_0 in memory! (compu

Ahora podemos realizar predicciones sobre el conjunto de test y sacar distintas métricas que midan su calidad.

In [20]:
prediccion = Model_dt.transform(datos_test)

Ahora vamos a cambiar el tipo de la columna "target" para evitar errores a la hora de obtener las métricas del ajuste.

In [21]:
prediccion= prediccion.withColumn("target", prediccion["target"].cast("double"))
prediccion = prediccion.select("target", "prediction")

In [22]:
metrics = MulticlassMetrics(prediccion.rdd.map(tuple))

/home/lu/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
22/07/05 01:07:12 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB


In [23]:
f_1 = metrics.fMeasure(1.0)
print(f'La métrica f_1 es: {f_1}')

22/07/05 01:07:13 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB


La métrica f_1 es: 0.8825744608010955


In [24]:
precision = metrics.precision(1.0)
print(f'La métrica Precisión es: {precision}')

La métrica Precisión es: 0.8026151930261519


In [25]:
recall = metrics.recall(1.0)
print(f'La métrica Recall es: {recall}')

La métrica Recall es: 0.9802281368821293


3) Bagging

In [26]:
dtrf = RandomForestClassifier(featuresCol = 'all_features', labelCol = 'target', numTrees = 4, maxDepth=7,
                                     impurity='gini', bootstrap=True, subsamplingRate=0.8, weightCol="target")
Model_dtrf = dtrf.fit(datos_entrenamiento)

22/07/05 01:07:16 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:07:18 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:07:20 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:07:22 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:07:24 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/07/05 01:07:25 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 01:07:27 WARN MemoryStore: Not enough space to cache rdd_556_0 in memory! (computed 229.9 MiB so far)
22/07/05 01:07:27 WARN BlockManager: Persisting block rdd_556_0 to disk instead.
22/07/05 01:09:15 WARN MemoryStore: Not enough space to cache rdd_556_0 in memory! (computed 229.9 MiB so far)


Ahora podemos realizar predicciones sobre el conjunto de test y sacar distintas métricas que midan su calidad.

In [27]:
prediccion = Model_dtrf.transform(datos_test)

Ahora vamos a cambiar el tipo de la columna "target" para evitar errores a la hora de obtener las métricas del ajuste.

In [28]:
prediccion= prediccion.withColumn("target", prediccion["target"].cast("double"))
prediccion = prediccion.select("target", "prediction")

In [29]:
metrics = MulticlassMetrics(prediccion.rdd.map(tuple))

22/07/05 01:10:01 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB


In [30]:
f_1 = metrics.fMeasure(1.0)
print(f'La métrica f_1 es: {f_1}')

22/07/05 01:10:02 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB


La métrica f_1 es: 0.19684991113562542


In [31]:
precision = metrics.precision(1.0)
print(f'La métrica Precisión es: {precision}')

La métrica Precisión es: 1.0


In [32]:
recall = metrics.recall(1.0)
print(f'La métrica Recall es: {recall}')

La métrica Recall es: 0.10917000883692475


4) Boosting

In [33]:
dtgb = GBTClassifier(featuresCol = 'all_features', labelCol = 'target', maxDepth=3, stepSize=0.004, maxIter=3)
Model_dtgb = dtgb.fit(datos_entrenamiento)

22/07/05 01:10:05 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:10:07 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:10:09 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:10:10 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/07/05 01:10:10 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 01:10:12 WARN MemoryStore: Not enough space to cache rdd_614_0 in memory! (computed 229.9 MiB so far)
22/07/05 01:10:12 WARN BlockManager: Persisting block rdd_614_0 to disk instead.
22/07/05 01:11:56 WARN MemoryStore: Not enough space to cache rdd_614_0 in memory! (computed 229.9 MiB so far)
22/07/05 01:11:56 WARN MemoryStore: Not enough space to cache rdd_614_0 in memory! (computed 97.1 MiB so far)
22/07/05 01:14:06 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 01:14:07 WARN MemoryStore: Not enough space to cache rdd_

Ahora podemos realizar predicciones sobre el conjunto de test y sacar distintas métricas que midan su calidad.

In [34]:
prediccion = Model_dtgb.transform(datos_test)

Ahora vamos a cambiar el tipo de la columna "target" para evitar errores a la hora de obtener las métricas del ajuste.

In [35]:
prediccion= prediccion.withColumn("target", prediccion["target"].cast("double"))
prediccion = prediccion.select("target", "prediction")

In [36]:
metrics = MulticlassMetrics(prediccion.rdd.map(tuple))

22/07/05 01:29:49 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB


In [37]:
f_1 = metrics.fMeasure(1.0)
print(f'La métrica f_1 es: {f_1}')

22/07/05 01:29:51 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB


La métrica f_1 es: 0.6627810158201498


In [38]:
precision = metrics.precision(1.0)
print(f'La métrica Precisión es: {precision}')

La métrica Precisión es: 0.49564134495641343


In [39]:
recall = metrics.recall(1.0)
print(f'La métrica Recall es: {recall}')

La métrica Recall es: 1.0


5) Support Vector Machines

In [40]:
svm = LinearSVC(featuresCol = 'all_features', labelCol = 'target',regParam=1)
Model_svm = svm.fit(datos_entrenamiento)

22/07/05 01:29:54 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:29:55 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:29:57 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:29:58 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:29:59 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:30:00 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:30:01 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:30:02 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:30:03 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:30:04 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:30:05 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:30:06 WARN DAGScheduler: Broadc

22/07/05 01:33:35 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:36 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:38 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:40 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:42 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:43 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:45 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:46 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:48 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:50 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:52 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:33:54 WARN DAGScheduler: Broadc

Ahora podemos realizar predicciones sobre el conjunto de test y sacar distintas métricas que midan su calidad.

In [41]:
prediccion = Model_svm.transform(datos_test)

Ahora vamos a cambiar el tipo de la columna "target" para evitar errores a la hora de obtener las métricas del ajuste.

In [42]:
prediccion= prediccion.withColumn("target", prediccion["target"].cast("double"))
prediccion = prediccion.select("target", "prediction")

In [43]:
metrics = MulticlassMetrics(prediccion.rdd.map(tuple))

22/07/05 01:34:21 WARN DAGScheduler: Broadcasting large task binary with size 22.0 MiB


In [44]:
f_1 = metrics.fMeasure(1.0)
print(f'La métrica f_1 es: {f_1}')

22/07/05 01:34:24 WARN DAGScheduler: Broadcasting large task binary with size 22.0 MiB


La métrica f_1 es: 0.012062726176115802


In [45]:
precision = metrics.precision(1.0)
print(f'La métrica Precisión es: {precision}')

La métrica Precisión es: 0.0062266500622665


In [46]:
recall = metrics.recall(1.0)
print(f'La métrica Recall es: {recall}')

La métrica Recall es: 0.19230769230769232


6) Naive Bayes

In [47]:
nb = NaiveBayes(featuresCol = 'all_features', labelCol = 'target',smoothing=1.0, modelType="gaussian")
Model_nb = nb.fit(datos_entrenamiento)

22/07/05 01:34:28 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 01:34:34 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB


Ahora podemos realizar predicciones sobre el conjunto de test y sacar distintas métricas que midan su calidad.

In [48]:
prediccion = Model_nb.transform(datos_test)

Ahora vamos a cambiar el tipo de la columna "target" para evitar errores a la hora de obtener las métricas del ajuste.

In [49]:
prediccion= prediccion.withColumn("target", prediccion["target"].cast("double"))
prediccion = prediccion.select("target", "prediction")

In [50]:
metrics = MulticlassMetrics(prediccion.rdd.map(tuple))

22/07/05 01:34:36 WARN DAGScheduler: Broadcasting large task binary with size 24.5 MiB


In [51]:
f_1 = metrics.fMeasure(1.0)
print(f'La métrica f_1 es: {f_1}')

22/07/05 01:39:26 WARN DAGScheduler: Broadcasting large task binary with size 24.5 MiB


La métrica f_1 es: 0.298370899352823


In [52]:
precision = metrics.precision(1.0)
print(f'La métrica Precisión es: {precision}')

La métrica Precisión es: 0.8325031133250311


In [53]:
recall = metrics.recall(1.0)
print(f'La métrica Recall es: {recall}')

La métrica Recall es: 0.18175638934203373


In [54]:
df

DataFrame[0: double, step: double, amount: double, oldbalanceOrg: double, newbalanceOrg: double, oldbalanceDest: double, newbalanceDest: double, isFlaggedFraud: double, target: int, type-: double, nameOrig-: double, nameDest-: double, type-encoded: vector, nameOrig-encoded: vector, nameDest-encoded: vector, all_features: vector]

In [55]:
datos_entrenamiento.schema["all_features"].metadata["ml_attr"]["num_attrs"]

143820

7) Multilayer Perceptron

Para ajustar el Multilayer Perceptron vamos a realizar una selección distinta de variables.

In [56]:
assembler = VectorAssembler(
    inputCols=['step', 'amount', 'oldbalanceOrg', 'newbalanceOrg', 'oldbalanceDest', 'newbalanceDest', 
              'isFlaggedFraud', 'type-encoded'],
    outputCol="all_features_MP"
)
df = assembler.transform(df)
datos_entrenamiento, datos_test = df.randomSplit([0.8,0.2])

In [57]:
mp = MultilayerPerceptronClassifier(featuresCol = 'all_features_MP', labelCol = 'target', 
                          maxIter = 500, solver = "l-bfgs", stepSize=0.005, layers=[11,6,4,2])
Model_mp = mp.fit(datos_entrenamiento)

22/07/05 01:44:36 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:39 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:41 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:42 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:43 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:45 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:47 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:49 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:50 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:52 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:53 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:44:55 WARN DAGScheduler: Broadc

22/07/05 01:49:25 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:27 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:28 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:30 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:31 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:33 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:34 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:36 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:38 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:39 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:40 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:49:42 WARN DAGScheduler: Broadc

22/07/05 01:54:49 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:54:52 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:54:53 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:54:56 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:54:57 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:54:59 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:55:00 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:55:03 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:55:04 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:55:06 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:55:08 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 01:55:10 WARN DAGScheduler: Broadc

22/07/05 02:00:10 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:11 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:14 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:15 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:16 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:18 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:19 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:21 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:22 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:23 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:25 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:00:26 WARN DAGScheduler: Broadc

22/07/05 02:04:57 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:04:59 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:01 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:02 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:03 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:05 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:06 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:08 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:09 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:11 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:12 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:05:14 WARN DAGScheduler: Broadc

22/07/05 02:10:40 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:43 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:44 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:46 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:47 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:50 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:52 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:54 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:55 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:57 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:10:59 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:11:01 WARN DAGScheduler: Broadc

Ahora podemos realizar predicciones sobre el conjunto de test y sacar distintas métricas que midan su calidad.

In [58]:
prediccion = Model_mp.transform(datos_test)

Ahora vamos a cambiar el tipo de la columna "target" para evitar errores a la hora de obtener las métricas del ajuste.

In [59]:
prediccion= prediccion.withColumn("target", prediccion["target"].cast("double"))
prediccion = prediccion.select("target", "prediction")

In [60]:
prediccion.show()

22/07/05 02:15:03 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB


+------+----------+
|target|prediction|
+------+----------+
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   1.0|       0.0|
|   0.0|       0.0|
|   1.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   1.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
|   0.0|       0.0|
+------+----------+
only showing top 20 rows



In [61]:
metrics = MulticlassMetrics(prediccion.rdd.map(tuple))

22/07/05 02:15:07 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB


In [62]:
f_1 = metrics.fMeasure(1.0)
print(f'La métrica f_1 es: {f_1}')

22/07/05 02:15:09 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB


La métrica f_1 es: 0.37493579866461224


In [63]:
precision = metrics.precision(1.0)
print(f'La métrica Precisión es: {precision}')

La métrica Precisión es: 0.23086654016445288


In [64]:
recall = metrics.recall(1.0)
print(f'La métrica Recall es: {recall}')

La métrica Recall es: 0.9972677595628415


Una vez hemos ajustado todos los modelos nos quedaremos con el que mejor métrica f_1 haya presentado (excluyendo al Perceptrón Multicapa de esta comparación pues lo hemos entrenado con distintos datos que los demás algoritmos) y procederemos a validarlo con el método de la Validación Cruzada con búsqueda de hiperparámetros. En este caso el algoritmo que escogeremos es pues: Árbol de Decisión

In [65]:
evaluator = MulticlassClassificationEvaluator(
    labelCol='target', 
    predictionCol="prediction")

In [66]:
paramGrid = (ParamGridBuilder()
.addGrid(dt.maxDepth, [3,7,9])
.addGrid(dt.impurity, ["gini", "entropy"]).build())

CV = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid,
                    evaluator=evaluator,numFolds=3)
CVModel = CV.fit(df)

22/07/05 02:15:12 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 02:15:16 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:15:18 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:15:19 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 02:15:20 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/07/05 02:15:21 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 02:15:23 WARN MemoryStore: Not enough space to cache rdd_2137_0 in memory! (computed 229.9 MiB so far)
22/07/05 02:15:23 WARN BlockManager: Persisting block rdd_2137_0 to disk instead.
22/07/05 02:17:24 WARN MemoryStore: Not enough space to cache rdd_2137_0 in memory! (computed 229.9 MiB so far)
22/07/05 02:18:54 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 02:18:55 WARN MemoryStore: Not enough space to cache rdd_2137_0 in memory! (c

22/07/05 03:59:36 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 03:59:37 WARN MemoryStore: Not enough space to cache rdd_2628_0 in memory! (computed 229.9 MiB so far)
22/07/05 04:00:56 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 04:00:57 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 04:00:59 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 04:01:00 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 04:01:02 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 04:01:03 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/07/05 04:01:04 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 04:01:05 WARN MemoryStore: Not enough space to cache rdd_2692_0 in memory! (computed 229.9 MiB so far)
22/07/05 04:01:05 WARN BlockManager: Persisting block rdd_2692_0 to disk instead.

22/07/05 05:47:38 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 05:47:39 WARN MemoryStore: Not enough space to cache rdd_3138_0 in memory! (computed 229.9 MiB so far)
22/07/05 05:49:21 WARN DAGScheduler: Broadcasting large task binary with size 24.8 MiB
22/07/05 05:49:23 WARN MemoryStore: Not enough space to cache rdd_3138_0 in memory! (computed 229.9 MiB so far)
22/07/05 05:51:01 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 05:51:03 WARN DAGScheduler: Broadcasting large task binary with size 20.9 MiB
22/07/05 05:51:06 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 05:51:07 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 05:51:09 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/07/05 05:51:10 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/07/05 05:51:11 WARN DAGScheduler: Broadcasting large task binary with size 24.

In [68]:
#Vemos cuales son los parámetros del mejor modelo
Model_dtbest = CVModel.bestModel
print("maxDepth:",Model_dtbest.getMaxDepth())
print("minInstancesPerNode:", Model_dtbest.getImpurity())

maxDepth: 9
minInstancesPerNode: gini


Por lo tanto ya hemos encontrado los hiperparámetros que buscábamos.

# APRENDIZAJE NO SUPERVISADO

Ahora analizaremos los siguientes algoritmos de Aprendizaje No Supervisado: K-means, Bisecting K-means, Gaussian Mixture Models y Power Iteration CLustering con el Dataset de segmentación de clientes que hemos explorado con anterioridad.

In [69]:
df = spark.read.csv('../../Datasets/segmentation data.csv',header=True, inferSchema=True)

Lo primero es aplicar la pipeline de preprocesado

In [70]:
pipeline_model_preprocesado = pipeline_preprocesado.fit(df)

In [71]:
df = pipeline_model_preprocesado.transform(df)

Ahora ya estamos listo para aplicar los algoritmos de Machine Learning

Lo primero que debemos hacer para poder introducir los datos a los modelos de Machine Learning de MLlib es procesarlos con un transformer llamado VectorAssembler. En el introduciremos las variables, menos el target, del conjunto de datos que deseamos introducir en el algoritmo y obtendremos una única de salida que daremos a los algoritmos.

In [72]:
assembler = VectorAssembler(
    inputCols=['Sex', 'Marital status', 'Age', 'Education', 'Income', 
              'Occupation', 'Settlement size'],
    outputCol="all_features"
)
df = assembler.transform(df)

1) K-means

In [73]:
km = KMeans(featuresCol = 'all_features', maxIter=40, k=5)
Model_km = km.fit(df)

In [74]:
prediccion = Model_km.transform(df)

Así podemos ver a que clúster ha sido asignado cada observación.

In [75]:
prediccion.select("ID","prediction").show()

+------------+----------+
|          ID|prediction|
+------------+----------+
|1.00000001E8|         2|
|1.00000002E8|         3|
|1.00000003E8|         4|
|1.00000004E8|         0|
|1.00000005E8|         3|
|1.00000006E8|         3|
|1.00000007E8|         0|
|1.00000008E8|         0|
|1.00000009E8|         3|
| 1.0000001E8|         0|
|1.00000011E8|         2|
|1.00000012E8|         2|
|1.00000013E8|         2|
|1.00000014E8|         4|
|1.00000015E8|         4|
|1.00000016E8|         4|
|1.00000017E8|         2|
|1.00000018E8|         0|
|1.00000019E8|         2|
| 1.0000002E8|         3|
+------------+----------+
only showing top 20 rows



A continuación, observamos los centroides de cada clúster

In [76]:
centers = Model_km.clusterCenters()
print(centers)

[array([3.10679612e-01, 4.90291262e-01, 4.02669903e+01, 1.00000000e+00,
       1.71313083e+05, 1.40776699e+00, 1.27184466e+00]), array([5.38461538e-01, 4.88461538e-01, 3.15730769e+01, 1.00000000e+00,
       6.98520923e+04, 3.84615385e-03, 1.92307692e-02]), array([4.67315716e-01, 4.89568846e-01, 3.52545202e+01, 1.00000000e+00,
       1.16592622e+05, 9.24895688e-01, 8.20584145e-01]), array([3.20754717e-01, 4.33962264e-01, 3.79703504e+01, 1.00000000e+00,
       1.40963210e+05, 1.12129380e+00, 1.14824798e+00]), array([5.74324324e-01, 5.67567568e-01, 3.12207207e+01, 1.00000000e+00,
       9.61739797e+04, 5.60810811e-01, 4.39189189e-01])]


2) Bisecting K-means

In [77]:
bkm = BisectingKMeans(featuresCol = 'all_features', maxIter=40, k=5, minDivisibleClusterSize=0.05)
Model_bkm = bkm.fit(df)

In [78]:
prediccion = Model_bkm.transform(df)

Así podemos ver a que clúster ha sido asignado cada observación.

In [79]:
prediccion.select("ID","prediction").show()

+------------+----------+
|          ID|prediction|
+------------+----------+
|1.00000001E8|         3|
|1.00000002E8|         4|
|1.00000003E8|         0|
|1.00000004E8|         4|
|1.00000005E8|         3|
|1.00000006E8|         3|
|1.00000007E8|         4|
|1.00000008E8|         4|
|1.00000009E8|         4|
| 1.0000001E8|         4|
|1.00000011E8|         2|
|1.00000012E8|         3|
|1.00000013E8|         2|
|1.00000014E8|         0|
|1.00000015E8|         1|
|1.00000016E8|         0|
|1.00000017E8|         3|
|1.00000018E8|         4|
|1.00000019E8|         2|
| 1.0000002E8|         3|
+------------+----------+
only showing top 20 rows



A continuación, observamos los centroides de cada clúster

In [80]:
centers = Model_km.clusterCenters()
print(centers)

[array([3.10679612e-01, 4.90291262e-01, 4.02669903e+01, 1.00000000e+00,
       1.71313083e+05, 1.40776699e+00, 1.27184466e+00]), array([5.38461538e-01, 4.88461538e-01, 3.15730769e+01, 1.00000000e+00,
       6.98520923e+04, 3.84615385e-03, 1.92307692e-02]), array([4.67315716e-01, 4.89568846e-01, 3.52545202e+01, 1.00000000e+00,
       1.16592622e+05, 9.24895688e-01, 8.20584145e-01]), array([3.20754717e-01, 4.33962264e-01, 3.79703504e+01, 1.00000000e+00,
       1.40963210e+05, 1.12129380e+00, 1.14824798e+00]), array([5.74324324e-01, 5.67567568e-01, 3.12207207e+01, 1.00000000e+00,
       9.61739797e+04, 5.60810811e-01, 4.39189189e-01])]


3) Gaussian Mixture Models

In [81]:
gm = GaussianMixture(featuresCol = 'all_features', maxIter=40, k=5)
Model_gm = gm.fit(df)

22/07/05 08:36:27 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/07/05 08:36:27 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [82]:
prediccion = Model_bkm.transform(df)

Así podemos ver a que clúster ha sido asignado cada observación.

In [83]:
prediccion.select("ID","prediction").show()

+------------+----------+
|          ID|prediction|
+------------+----------+
|1.00000001E8|         3|
|1.00000002E8|         4|
|1.00000003E8|         0|
|1.00000004E8|         4|
|1.00000005E8|         3|
|1.00000006E8|         3|
|1.00000007E8|         4|
|1.00000008E8|         4|
|1.00000009E8|         4|
| 1.0000001E8|         4|
|1.00000011E8|         2|
|1.00000012E8|         3|
|1.00000013E8|         2|
|1.00000014E8|         0|
|1.00000015E8|         1|
|1.00000016E8|         0|
|1.00000017E8|         3|
|1.00000018E8|         4|
|1.00000019E8|         2|
| 1.0000002E8|         3|
+------------+----------+
only showing top 20 rows



A continuación, observamos los centroides de cada clúster

In [84]:
centers = Model_km.clusterCenters()
print(centers)

[array([3.10679612e-01, 4.90291262e-01, 4.02669903e+01, 1.00000000e+00,
       1.71313083e+05, 1.40776699e+00, 1.27184466e+00]), array([5.38461538e-01, 4.88461538e-01, 3.15730769e+01, 1.00000000e+00,
       6.98520923e+04, 3.84615385e-03, 1.92307692e-02]), array([4.67315716e-01, 4.89568846e-01, 3.52545202e+01, 1.00000000e+00,
       1.16592622e+05, 9.24895688e-01, 8.20584145e-01]), array([3.20754717e-01, 4.33962264e-01, 3.79703504e+01, 1.00000000e+00,
       1.40963210e+05, 1.12129380e+00, 1.14824798e+00]), array([5.74324324e-01, 5.67567568e-01, 3.12207207e+01, 1.00000000e+00,
       9.61739797e+04, 5.60810811e-01, 4.39189189e-01])]


4) Power Iteration Clustering

Como las aplicaciones de este algoritmo son diferentes a las de los tres algoritmos de clustering que acabamos de ver realizaremos un ejemplo más sencillo exclusivamente para el algoritmo Power Iteration Clustering. Además, este algoritmo presenta el incoviente de que todavía no está completamente desarrollado bajo la estructura de Estimator y Transformer y por lo tanto la forma de utilizarlo es distinta.

In [87]:
datos = [(1, 0, 0.5),

        (2, 0, 0.5), (2, 1, 0.7),

        (3, 0, 0.5), (3, 1, 0.7), (3, 2, 0.9),

        (4, 0, 0.5), (4, 1, 0.7), (4, 2, 0.9), (4, 3, 1.1),

        (5, 0, 0.5), (5, 1, 0.7), (5, 2, 0.9), (5, 3, 1.1), (5, 4, 1.3)]
#Creamos un DataFrame de Spark
df = spark.createDataFrame(datos).toDF("src", "dst", "weight")

In [88]:
pic = PowerIterationClustering(k=2, weightCol="weight", maxIter=20)
# Este paso hace falta hacerlo para usar el algoritmo ya que todavía no tiene bien desarrolladas sus clases de 
# Estimator y Transformer.
assignments = pic.assignClusters(df)
assignments.sort(assignments.id).show(truncate=False)

+---+-------+
|id |cluster|
+---+-------+
|0  |0      |
|1  |0      |
|2  |0      |
|3  |0      |
|4  |0      |
|5  |1      |
+---+-------+

